IMPORT THE REQUIRED LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


LOADING THE DATASET

In [2]:
df = pd.read_csv('/home/izy/Documents/Project/Cardiovascular Disease Risk Prediction Model/venv/data/healthcare_synthetic_data.csv')

GET THE IMPORTANT INFORMATION ON THE DATA

In [3]:
df.head()

,Patient_ID,Age,Gender,Height_cm,Weight_kg,BMI,Systolic_BP,Diastolic_BP,Cholesterol_Total,Cholesterol_LDL,Cholesterol_HDL,Fasting_Blood_Sugar,Smoking_Status,Alcohol_Consumption,Physical_Activity_Level,Family_History,Stress_Level,Sleep_Hours,Heart_Disease_Risk
0,PID-00001,60,0,146.9,51.3,23.8,140,89,217,151,52,83,0,1,3,0,1,8,0
1,PID-00002,53,0,161.8,76.6,29.3,128,81,203,119,38,116,0,0,1,0,7,9,0
2,PID-00003,62,1,174.7,92.4,30.3,141,100,173,124,45,90,0,0,0,0,1,7,1
3,PID-00004,73,1,173.3,68.9,22.9,136,96,193,117,45,81,0,0,1,0,2,7,1
4,PID-00005,52,1,178.6,79.8,25.0,122,80,236,153,41,79,0,1,2,0,2,6,0


In [4]:
df.columns

Index(['Patient_ID', 'Age', 'Gender', 'Height_cm', 'Weight_kg', 'BMI',
       'Systolic_BP', 'Diastolic_BP', 'Cholesterol_Total', 'Cholesterol_LDL',
       'Cholesterol_HDL', 'Fasting_Blood_Sugar', 'Smoking_Status',
       'Alcohol_Consumption', 'Physical_Activity_Level', 'Family_History',
       'Stress_Level', 'Sleep_Hours', 'Heart_Disease_Risk'],
      dtype='object')

In [8]:
df = df.drop(columns='Patient_ID')

In [9]:
df.head()

,Age,Gender,Height_cm,Weight_kg,BMI,Systolic_BP,Diastolic_BP,Cholesterol_Total,Cholesterol_LDL,Cholesterol_HDL,Fasting_Blood_Sugar,Smoking_Status,Alcohol_Consumption,Physical_Activity_Level,Family_History,Stress_Level,Sleep_Hours,Heart_Disease_Risk
0,60,0,146.9,51.3,23.8,140,89,217,151,52,83,0,1,3,0,1,8,0
1,53,0,161.8,76.6,29.3,128,81,203,119,38,116,0,0,1,0,7,9,0
2,62,1,174.7,92.4,30.3,141,100,173,124,45,90,0,0,0,0,1,7,1
3,73,1,173.3,68.9,22.9,136,96,193,117,45,81,0,0,1,0,2,7,1
4,52,1,178.6,79.8,25.0,122,80,236,153,41,79,0,1,2,0,2,6,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      15000 non-null  int64  
 1   Gender                   15000 non-null  int64  
 2   Height_cm                15000 non-null  float64
 3   Weight_kg                15000 non-null  float64
 4   BMI                      15000 non-null  float64
 5   Systolic_BP              15000 non-null  int64  
 6   Diastolic_BP             15000 non-null  int64  
 7   Cholesterol_Total        15000 non-null  int64  
 8   Cholesterol_LDL          15000 non-null  int64  
 9   Cholesterol_HDL          15000 non-null  int64  
 10  Fasting_Blood_Sugar      15000 non-null  int64  
 11  Smoking_Status           15000 non-null  int64  
 12  Alcohol_Consumption      15000 non-null  int64  
 13  Physical_Activity_Level  15000 non-null  int64  
 14  Family_History        

We are going to convert the float64 datatype into int64 datatype

In [13]:
df['BMI'] = df['BMI'].astype(int)
df['Height_cm'] = df['Height_cm'].astype(int)
df['Weight_kg'] = df['Weight_kg'].astype(int)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Age                      15000 non-null  int64
 1   Gender                   15000 non-null  int64
 2   Height_cm                15000 non-null  int64
 3   Weight_kg                15000 non-null  int64
 4   BMI                      15000 non-null  int64
 5   Systolic_BP              15000 non-null  int64
 6   Diastolic_BP             15000 non-null  int64
 7   Cholesterol_Total        15000 non-null  int64
 8   Cholesterol_LDL          15000 non-null  int64
 9   Cholesterol_HDL          15000 non-null  int64
 10  Fasting_Blood_Sugar      15000 non-null  int64
 11  Smoking_Status           15000 non-null  int64
 12  Alcohol_Consumption      15000 non-null  int64
 13  Physical_Activity_Level  15000 non-null  int64
 14  Family_History           15000 non-null  int64
 15  St

In [15]:
df.describe

<bound method NDFrame.describe of        Age  Gender  Height_cm  Weight_kg  BMI  Systolic_BP  Diastolic_BP  \
0       60       0        146         51   23          140            89   
1       53       0        161         76   29          128            81   
2       62       1        174         92   30          141           100   
3       73       1        173         68   22          136            96   
4       52       1        178         79   25          122            80   
...    ...     ...        ...        ...  ...          ...           ...   
14995   44       1        166         81   29          135            85   
14996   69       0        154         61   26          124            79   
14997   70       1        178         88   27          129            93   
14998   44       0        157         60   24          122            84   
14999   53       1        170         72   25          150           101   

       Cholesterol_Total  Cholesterol_LDL  Cholestero